In [1]:
from __future__ import print_function
from ipywidgets import widgets, interact
from IPython.display import Audio, display

In [8]:
import music21 as m21
import random
import threading
from collections import OrderedDict

In [9]:
def chords_to_file(chord_list, tempo=80, mid_filename='tmp.mid', wav_filename='tmp.wav'):
    s = m21.stream.Stream()
    s.append(m21.tempo.MetronomeMark(number=tempo))
    for c in chord_list:
        s.append(m21.chord.Chord(c))
        
    mf = m21.midi.translate.streamToMidiFile(s)
    mf.open(mid_filename, 'wb')
    mf.write()
    mf.close()

    ! timidity -Ow -o {wav_filename} {mid_filename} > /dev/null
    
    return wav_filename

In [30]:
Imaj7 = m21.chord.Chord(['c4', 'e4', 'g4', 'b4'])
IIm7 = m21.chord.Chord(['d4', 'f4', 'a4', 'c5'])
IIIm7 = m21.chord.Chord(['e4', 'g4', 'b4', 'd5'])
IVmaj7 = m21.chord.Chord(['f4', 'a4', 'c5', 'e5'])
V7 = m21.chord.Chord(['g4', 'b4', 'd5', 'f5'])
VIm7 = m21.chord.Chord(['a4', 'c5', 'e5', 'g5'])

scale_chords = OrderedDict([
    ('Imaj7', Imaj7),
    ('IIm7', IIm7),
    ('IIIm7', IIIm7),
    ('IVmaj7', IVmaj7),
    ('V7', V7),
    ('VIm7', VIm7)
])

secondary_dominants = OrderedDict([
    ('V/IV', m21.chord.Chord(['c4', 'e4', 'g4', 'b-4'])),
    ('V/V', m21.chord.Chord(['d4', 'f#4', 'a4', 'c5'])),
    ('V/VI', m21.chord.Chord(['e4', 'g#4', 'b4', 'd5'])),
    ('V/II', m21.chord.Chord(['a4', 'c#5', 'e5', 'g5'])),
])

In [31]:
## Lvl 1: Two chords, first one is Imaj7

def lvl_1_update():
    second_chord_name, second_chord = random.choice(scale_chords.items())
    return second_chord_name, [Imaj7, second_chord]

lvl_1 = {
    'update': lvl_1_update,
    'options': scale_chords.keys()
}

def lvl_2_update():
    second_chord_name, second_chord = random.choice(scale_chords.items() + secondary_dominants.items())
    return second_chord_name, [Imaj7, second_chord]

lvl_2 = {
    'update': lvl_2_update,
    'options': scale_chords.keys() + secondary_dominants.keys()
}
    

In [34]:
lvl = lvl_2

set_length = 10

# Response

correct_ans_count = 0
cur_correct_ans = None
cur_ans = None
count = 0

# Gui
non_option = 'Select...'
chord_select = widgets.Select(
    options=[non_option] + lvl['options']
)

def response(option):
    global correct_ans_count, cur_correct_ans, count
    if option == non_option:
        return
    
    if count == set_length:
        s.close()
        print('Result: {}/{} ({:.2f}%)'.format(correct_ans_count, 
                                               set_length, correct_ans_count / float(set_length) * 100))

    if option == cur_correct_ans:
        correct_ans_count += 1
        print('Correct!')
    else:
        print(cur_correct_ans)
        
    count += 1
        
    second_chord_name, chord_prog = lvl['update']()
    wav_filename = chords_to_file(chord_prog)

    cur_correct_ans = second_chord_name

    a = Audio(wav_filename)
    display(a)
    s.value = non_option
        
    
s = widgets.interactive(response, option=chord_select)

second_chord_name, chord_prog = lvl['update']()
wav_filename = chords_to_file(chord_prog)

cur_correct_ans = second_chord_name

a = Audio(wav_filename)

display(s)
display(a)

Result: 5/10 (50.00%)
Correct!
